In [7]:
include("../src/Conduction.jl");
using Plots, LaTeXStrings, QuadGK;

semiconductor = Conduction.Semiconductor(
4.34e7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
0.1 * 1.6e-19, # Mode effect of the phonons (J)
1e36, # intrinsic semiconductor's density (cm^-3)
0, # Doping states' density (cm^-3)
0.1 * 1.6e-19, # Energy to a vacant target site (J)
-5.3e1, # Field (V.cm^-1)
1e13, # Base electron jump rate
-48, # Fermi level (J)
3.87, # Intrinsic semiconductor's gaussian width (J)
2.7, # Doping states' gaussian width (J)
2.7, # Amount of disorder
150e-7
);

T = 300;

In [8]:
p() = quadgk(
    r -> Conduction.DOS(semiconductor, r, T) * Conduction.F(semiconductor, r, T) * Conduction.k * T,
    -Inf,
    Inf
)[1];

p()

2.5471667776391716e18

In [9]:
Conduction.overallEin(semiconductor, Conduction.RnnVRH, 20)

LoadError: MethodError: no method matching overallEin(::Main.Conduction.Semiconductor, ::typeof(Main.Conduction.RnnVRH), ::Int64)
[0mClosest candidates are:
[0m  overallEin(::Main.Conduction.Semiconductor, ::Function, ::Any, [91m::Real[39m) at /home/lilifaj/Workspace/Thesis/Conduction/src/Conduction.jl:208